In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 50) # 2 Input noses, 50 in middle layers
        self.fc2 = nn.Linear(50, 1) # 50 middle layer, 1 output nodes
        self.rl1 = nn.ReLU()
        self.rl2 = nn.ReLU()
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.rl1(x)
        x = self.fc2(x)
        x = self.rl2(x)
        return x

In [3]:
net = Net()
net.cuda()

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.1)

trainingdataX = [[[0.01, 0.01], [0.01, 0.90], [0.90, 0.01], [0.95, 0.95]], [[0.02, 0.03], [0.04, 0.95], [0.97, 0.02], [0.96, 0.95]]]
trainingdataY = [[[0.01], [0.90], [0.90], [0.01]], [[0.04], [0.97], [0.98], [0.1]]]
NumEpoches = 10000
for epoch in range(NumEpoches):

    running_loss = 0.0
    for i, data in enumerate(trainingdataX, 0):
        inputs = data
        labels = trainingdataY[i]
        inputs = Variable(torch.FloatTensor(inputs)).cuda()
        labels = Variable(torch.FloatTensor(labels)).cuda()
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()        
        optimizer.step()
        running_loss += loss.item()
    if epoch % 1000 == 0:
        print("loss: ", running_loss)
print("Finished training...")
print(net(Variable(torch.FloatTensor(trainingdataX[0])).cuda()))

loss:  0.7208119034767151
loss:  0.19625259935855865
loss:  0.0703129991889
loss:  0.017746583558619022
loss:  0.004908343777060509
loss:  0.0025783070595934987
loss:  0.002181260264478624
loss:  0.0020784364314749837
loss:  0.0020223130704835057
loss:  0.0019770973594859242
Finished training...
tensor([[0.0132],
        [0.9283],
        [0.9261],
        [0.0600]], device='cuda:0', grad_fn=<ThresholdBackward0>)


In [4]:
net.eval()


testingdataX = [[[0.01, 0.01], [0.01, 0.90], [0.90, 0.01], [0.95, 0.95]], [[0.02, 0.03], [0.04, 0.95], [0.97, 0.02], [0.96, 0.95]]]
ttestingdataY = [[[0.01], [0.90], [0.90], [0.01]], [[0.04], [0.97], [0.98], [0.1]]]

running_loss = 0.0
for i, data in enumerate(trainingdataX, 0):
    inputs = data
    labels = trainingdataY[i]
    inputs = Variable(torch.FloatTensor(inputs)).cuda()
    labels = Variable(torch.FloatTensor(labels)).cuda()
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    running_loss += loss.item()
print("loss: ", running_loss)
        
print("Finished testing...")
print(net(Variable(torch.FloatTensor(trainingdataX[0])).cuda()))

loss:  0.0019134845351800323
Finished testing...
tensor([[0.0132],
        [0.9283],
        [0.9261],
        [0.0600]], device='cuda:0', grad_fn=<ThresholdBackward0>)
